## LIB

In [2]:
# Basic lib
import pandas as pd
import numpy as np
import os
import sys

# Custom lib
from src.loader import Loader

## Statics variable

In [9]:
DATA_DIR = '.'
FILE_NAME = 'sample_data.csv'
DATA_FILE_PATH = os.path.join(DATA_DIR, FILE_NAME)
DATA_FILE_PATH

'.\\sample_data.csv'

## Data load

## Preprocess

## EDA

## Inference

## Evaluation